In [75]:
import pandas as pd
from lifelines import CoxPHFitter
import numpy as np

# Load the dataset
data = pd.read_csv("data.csv")

data.head()



,Refrigerator ID,Age,Maintenance History,Usage Patterns,Environmental Conditions,Purchase Date,Failure Date
0,1,24,Good,Low,Normal,2019-01-01,2021-06-30
1,2,36,Average,Medium,Humid,2018-05-15,2022-01-10
2,3,48,Good,High,Extreme,2020-03-12,-
3,4,60,Poor,Medium,Normal,2017-11-20,2019-09-05
4,5,30,Good,Low,Normal,2019-08-05,-


In [76]:

data = data.replace('-', np.nan)

data = pd.get_dummies(data, columns=["Maintenance History", "Usage Patterns", "Environmental Conditions"])

# Convert date columns to datetime format
date_columns = ['Purchase Date', 'Failure Date']
data[date_columns] = data[date_columns].apply(pd.to_datetime)


# Create the Event column
data['Event'] = 1  # Set all records as events initially
data.loc[data['Failure Date'].isnull(), 'Event'] = 0  # Set censored records as non-events
data['Survival Time'] = (data['Failure Date'] - data['Purchase Date']).dt.total_seconds() / (24 * 60 * 60)

data.loc[data['Survival Time'].isnull(), 'Survival Time'] = 0  # Set censored records as non-events

data = data[['Maintenance History_Average', 'Maintenance History_Good',
       'Maintenance History_Poor', 'Usage Patterns_High', 'Usage Patterns_Low',
       'Usage Patterns_Medium', 'Environmental Conditions_Extreme',
       'Environmental Conditions_Humid', 'Environmental Conditions_Normal',
       'Event', 'Survival Time']]

for col in data.columns:
     
     if col=='Event':
          data[col] = data[col].astype('bool')
     else:
          data[col] = data[col].astype(int)

correlation_matrix = data.corr().abs()
upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
highly_correlated_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
data.drop(highly_correlated_features, axis=1, inplace=True)

# Apply ridge regression
cph = CoxPHFitter(penalizer=0.5)  # Adjust the penalizer value as needed
cph.fit(data, duration_col='Survival Time', event_col='Event', show_progress=True)

# Print model summary
print(cph.summary)








/var/folders/pg/fkh_0yvj47z_qzwskm5jffvh0000gn/T/ipykernel_3859/2094269161.py:31: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [74]:
data

,Maintenance History_Average,Maintenance History_Good,Maintenance History_Poor,Usage Patterns_High,Usage Patterns_Low,Usage Patterns_Medium,Environmental Conditions_Extreme,Environmental Conditions_Humid,Environmental Conditions_Normal,Event,Survival Time
0,0,1,0,0,1,0,0,0,1,True,911
1,1,0,0,0,0,1,0,1,0,True,1336
2,0,1,0,1,0,0,1,0,0,False,0
3,0,0,1,0,0,1,0,0,1,True,654
4,0,1,0,0,1,0,0,0,1,False,0
